In [75]:
#|default_exp losses
#|export
import torch
import numpy as np
import pandas as pd
from tsai.basics import *


# Custom losses

In [76]:
#|export
class Loss:
    def __init__(self, ranges, weights, device):
        self.device = device
        self.ranges = torch.Tensor(ranges).to(device)
        self.weights = torch.Tensor(weights).to(device)

    def set_weights(self, weights):
        self.weights = torch.Tensor(weights).to(device)

    
    def weighted_loss_tensor(self, target):
        batch, variables, horizon = target.shape # Shape (32, 4, 6)
        variable, range, interval = self.ranges.shape # Shape (4, 4, 2)

        target_shaped = torch.reshape(target, (batch, variables, 1, horizon)) # Shape (32, 4, 6) -> (32, 4, 1, 6)
        ranges_shaped = torch.reshape(self.ranges, (variable, range, 1, interval)) # Shape (4, 4, 2) -> (4, 4, 1, 2)

        weights_tensor = (1 + ((ranges_shaped[..., 0] <= target_shaped) * (target_shaped <= ranges_shaped[...,1]))).float().to(self.device)
        
        return torch.einsum('r,bvrh->bvh', self.weights, weights_tensor)

In [77]:
#|export
class wMSELoss(nn.Module, Loss):
    def __init__(self, ranges, weights, device):
        nn.Module.__init__(self)
        Loss.__init__(self ,ranges, weights, device)

    
    def forward(self, y_pred, y_true):
        return torch.mean(self.weighted_loss_tensor(y_true) * (y_pred - y_true) ** 2).cpu()

In [78]:
#|export
class wMAELoss(nn.Module, Loss):
    def __init__(self, ranges, weights, device):
        nn.Module.__init__(self)
        Loss.__init__(self ,ranges, weights, device)

    
    def forward(self, y_pred, y_true):
        return torch.mean(self.weighted_loss_tensor(y_true) * torch.abs(y_pred - y_true)).cpu()

In [79]:
#|export

class LossMetrics:
    def __init__(self, loss_func:Loss):
        self.loss_func = loss_func

    def loss_call(self, input, target, weight_idx):
        loss_copy = deepcopy(self.loss_func)

        weights = torch.zeros(len(loss_copy.weights))
        weights[weight_idx] = loss_copy.weights[weight_idx]
        loss_copy.set_weights(weights)
        
        return loss_copy.forward(input, target)

    def loss_low(self, input, target):
        return self.loss_call(input, target, 0)

    def loss_moderate(self, input, target):
        return self.loss_call(input, target, 1)

    def loss_elevated(self, input, target):
        return self.loss_call(input, target, 2)

    def loss_high(self, input, target):
        return self.loss_call(input, target, 3)

    def metrics(self):
        return [self.loss_low, self.loss_moderate, self.loss_elevated, self.loss_high]

In [80]:
weights = [0.1, 0.2, 0.3, 0.4]
ranges = [[[0, 1], [0, 1], [0, 1], [0, 1]],
            [[0, 1], [0, 1], [0, 1], [0, 1]],
            [[0, 1], [0, 1], [0, 1], [0, 1]],
            [[0, 1], [0, 1], [0, 1], [0, 1]]]

loss = wMAELoss(ranges, weights, torch.device('cuda:0'))
metrics = LossMetrics(wMAELoss)
loss_metrics = LossMetrics(loss)

# Get metrics functions
metrics = loss_metrics.metrics()

input = torch.rand(32, 4, 6).to('cuda:0')
target = torch.rand(32, 4, 6).to('cuda:0')

print(loss.forward(input, target))
for metric in metrics:
    print(metric(input, target))

tensor(0.6679)
tensor(0.0668)
tensor(0.1336)
tensor(0.2004)
tensor(0.2671)
